In [1]:
import time, os, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt
# import sqlite3
from pandasql import sqldf
import seaborn as sns
pysqldf = lambda q: sqldf(q, globals())

np.random.seed(42)
from pandas import get_dummies
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold,\
cross_val_score, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc,\
classification_report, make_scorer, recall_score, mean_squared_error,\
precision_recall_curve, precision_score, auc
from sklearn.metrics import roc_auc_score as ras
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,\
StandardScaler, MinMaxScaler, normalize
from imblearn.over_sampling import SMOTE, RandomOverSampler
import warnings
warnings.filterwarnings('ignore')

## HPSA

In [2]:
# Load dataset for all Primary Care HPSA destinations with one record per facility
df_hpsa_pc = pd.read_csv("data/hpsa/BCD_HPSA_FCT_DET_PC.csv")

In [3]:
df_hpsa_pc[["HPSA Component Name", "HPSA Geography Identification Number", "Common State County FIPS Code"]]

,HPSA Component Name,HPSA Geography Identification Number,Common State County FIPS Code
0,"Census Tract 48, Lucas County, Ohio",39095004800,39095
1,"Census Tract 46, Lucas County, Ohio",39095004600,39095
2,"Census Tract 47.01, Lucas County, Ohio",39095004701,39095
3,Noble Correctional Institution,POINT,39121
4,EDGERTON FAMILY HEALTH CENTER,POINT,39171
...,...,...,...
66853,Lipscomb,48295,48295
66854,SEMINOLE FAMILY MEDICAL CLINIC,POINT,48165
66855,Schleicher,48413,48413
66856,Baylor,48023,48023


In [4]:
df_hpsa_pc = df_hpsa_pc[(df_hpsa_pc['Designation Type']=="HPSA Population") | 
                        (df_hpsa_pc["Designation Type"]=="Geographic HPSA") |
                        (df_hpsa_pc["Designation Type"]=="High Needs Geographic HPSA")
                       ]

len(df_hpsa_pc)

59589

In [5]:
# filter to the targe years

df_hpsa_pc['HPSA Designation Year'] = df_hpsa_pc['HPSA Designation Date'].str[-4:].astype(int)
df_hpsa_pc['HPSA Designation Last Update Year'] = df_hpsa_pc['HPSA Designation Last Update Date'].str[-4:].astype(int)

df_hpsa_pc = df_hpsa_pc[df_hpsa_pc['HPSA Designation Last Update Year'] >= 2020]

df_hpsa_pc = df_hpsa_pc[df_hpsa_pc["HPSA Status"]=="Designated"]

In [6]:
df_hpsa_pc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16115 entries, 0 to 66857
Data columns (total 68 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   HPSA Name                                                                 16115 non-null  object 
 1   HPSA ID                                                                   16115 non-null  object 
 2   Designation Type                                                          16115 non-null  object 
 3   HPSA Discipline Class                                                     16115 non-null  object 
 4   HPSA Score                                                                16115 non-null  int64  
 5   PC MCTA Score                                                             16084 non-null  float64
 6   Primary State Abbreviation                                         

In [7]:
df_hpsa_pc.rename(columns={"Common State County FIPS Code":"CountyFIPS", "HPSA Geography Identification Number":"LocationID"}, inplace=True)

In [8]:
df_hpsa_county_scores = df_hpsa_pc.groupby("CountyFIPS")["HPSA Score"].mean().reset_index()
df_hpsa_county_scores.head()

,CountyFIPS,HPSA Score
0,01001,15.0
1,01003,14.0
2,01005,20.0
3,01007,10.0
4,01009,7.0


In [9]:
df_hpsa_census_scores = df_hpsa_pc.groupby("LocationID")["HPSA Score"].mean().reset_index()
df_hpsa_census_scores.head()

,LocationID,HPSA Score
0,01001,15.0
1,0100390207,14.0
2,0100390963,14.0
3,0100392754,14.0
4,0100393024,14.0


In [10]:
# df_hpsa_county_status = df_hpsa_pc.groupby("CountyFIPS")["HPSA Status"].value_counts()

# most_common_strings = df_hpsa_county_status.sort_values(ascending=False)

# # Get the index (most frequent string) for each group
# df_hpsa_county_status = most_common_strings.

# df_hpsa_county_status


In [11]:
df_hpsa_county_scores["CountyFIPS"] = pd.to_numeric(df_hpsa_county_scores["CountyFIPS"], errors='coerce')
df_hpsa_census_scores["LocationID"] = pd.to_numeric(df_hpsa_census_scores["LocationID"], errors='coerce')

## Places

In [12]:
df_places = pd.read_csv('data/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2023_release_20240418.csv')

In [13]:
df_places.head()

,Year,StateAbbr,StateDesc,CountyName,CountyFIPS,LocationName,DataSource,Category,Measure,Data_Value_Unit,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,2021,AL,Alabama,Baldwin,1003,1003011300,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,29.8,40.9,4487,POINT (-87.87730464 30.43411562),1003011300,HLTHOUT,OBESITY,CrdPrv,Obesity
1,2021,AL,Alabama,Chambers,1017,1017954000,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,38.0,54.3,6669,POINT (-85.46010298 32.86128629),1017954000,HLTHOUT,OBESITY,CrdPrv,Obesity
2,2021,AL,Alabama,Cleburne,1029,1029959600,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,31.0,47.8,4202,POINT (-85.56228862 33.69258271),1029959600,HLTHOUT,OBESITY,CrdPrv,Obesity
3,2021,AL,Alabama,Covington,1039,1039962000,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,...,NaN,3.8,4.6,3917,POINT (-86.4651625 31.34674334),1039962000,HLTHOUT,STROKE,CrdPrv,Stroke
4,2021,AL,Alabama,Fayette,1057,1057020400,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,...,NaN,28.4,44.1,3550,POINT (-87.60702531 33.64745531),1057020400,HLTHOUT,OBESITY,CrdPrv,Obesity


In [14]:
df_places["Year"].value_counts()

Year
2021    1976988
2020     578125
Name: count, dtype: int64

In [15]:
df_places[["CountyName","CountyFIPS","LocationID"]].dtypes

CountyName    object
CountyFIPS     int64
LocationID     int64
dtype: object

In [16]:
df_healthout_county = df_places.groupby(["CountyFIPS", "Category", "MeasureId"])[["Data_Value"]].mean().reset_index()

# Pivot table with desired format
pivoted_df = df_healthout_county.pivot_table(values='Data_Value', index='CountyFIPS', columns='MeasureId')

# Convert pivot table to DataFrame (if desired)
pivoted_df_county = pivoted_df.reset_index()

pivoted_df_county

MeasureId,CountyFIPS,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,CHD,...,MAMMOUSE,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION
0,1001,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,83.750000,6.141667,...,77.691667,18.491667,16.491667,40.241667,12.866667,4.325000,38.358333,3.291667,16.125000,5.341667
1,1003,9.458065,31.293548,16.025806,37.838710,80.809677,7.670968,9.929032,84.980645,6.922581,...,77.829032,16.564516,16.219355,38.158065,12.422581,3.754839,34.493548,3.329032,12.587097,4.622581
2,1005,17.244444,34.655556,12.111111,47.322222,81.455556,6.755556,11.755556,80.944444,8.733333,...,77.711111,20.477778,25.055556,43.566667,17.388889,7.722222,42.411111,5.100000,26.544444,9.722222
3,1007,13.875000,31.125000,15.125000,40.075000,78.750000,6.475000,10.575000,82.050000,6.875000,...,74.900000,19.575000,18.375000,39.400000,14.500000,4.950000,39.400000,3.625000,20.025000,6.175000
4,1009,13.122222,31.922222,15.011111,38.544444,79.044444,7.200000,10.400000,82.400000,7.266667,...,73.122222,18.755556,18.111111,38.411111,14.244444,4.522222,36.966667,3.500000,17.811111,5.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,14.283333,20.925000,18.366667,27.091667,70.916667,5.783333,9.616667,79.650000,4.650000,...,64.341667,16.475000,10.675000,34.225000,10.566667,3.050000,30.091667,2.391667,15.725000,4.125000
3138,56039,10.050000,19.075000,21.075000,23.350000,69.850000,5.875000,8.750000,84.575000,3.925000,...,74.500000,12.850000,7.675000,21.875000,7.925000,2.175000,26.450000,1.950000,8.725000,2.925000
3139,56041,13.933333,22.233333,16.433333,28.600000,71.466667,6.033333,9.866667,80.233333,5.200000,...,65.200000,16.333333,12.100000,34.433333,11.466667,3.566667,35.333333,2.700000,16.333333,4.500000
3140,56043,14.000000,27.733333,14.766667,32.800000,78.300000,7.933333,9.800000,80.333333,6.933333,...,66.833333,14.766667,14.666667,34.466667,12.233333,3.966667,30.166667,3.433333,16.366667,5.100000


In [17]:
df_healthout_census = df_places.groupby(["LocationID", "Category", "MeasureId"])[["Data_Value"]].mean().reset_index()

# Pivot table with desired format
pivoted_df = df_healthout_census.pivot_table(values='Data_Value', index='LocationID', columns='MeasureId')

# Convert pivot table to DataFrame (if desired)
pivoted_df_census = pivoted_df.reset_index()

pivoted_df_census

MeasureId,LocationID,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,CHD,...,MAMMOUSE,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION
0,1001020100,10.2,30.1,15.4,37.7,79.0,6.6,10.3,83.5,6.0,...,76.8,18.3,15.5,38.7,12.5,3.9,36.9,3.0,14.6,4.7
1,1001020200,13.7,28.8,13.9,42.2,78.8,5.3,11.3,84.6,5.4,...,81.0,19.4,17.8,45.0,13.2,5.0,43.4,3.6,19.4,6.5
2,1001020300,11.4,30.1,15.1,38.5,79.1,6.6,10.6,82.6,6.1,...,76.9,18.9,16.4,39.4,12.8,4.1,38.1,3.3,16.2,5.2
3,1001020400,7.9,32.0,15.0,38.3,81.7,8.4,9.3,85.9,6.5,...,75.5,15.2,14.2,34.8,10.9,3.0,33.4,3.1,10.9,3.6
4,1001020500,8.4,26.5,16.3,33.7,76.6,6.2,9.8,86.0,4.7,...,77.8,16.6,12.0,36.3,10.1,2.9,35.7,2.4,11.1,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72332,56043000200,11.8,28.2,14.8,33.1,78.7,8.2,9.6,81.7,6.8,...,67.7,14.0,13.8,34.2,11.8,3.7,29.8,3.3,14.4,4.5
72333,56043000301,15.8,25.1,16.0,29.9,75.8,7.0,10.0,79.5,6.1,...,65.9,15.7,13.6,34.1,11.9,3.7,30.7,3.1,17.1,5.0
72334,56043000302,14.4,29.9,13.5,35.4,80.4,8.6,9.8,79.8,7.9,...,66.9,14.6,16.6,35.1,13.0,4.5,30.0,3.9,17.6,5.8
72335,56045951100,12.9,26.4,16.5,33.0,76.5,7.5,9.7,79.9,6.7,...,60.7,15.2,14.2,36.0,12.6,4.1,33.9,3.3,17.7,5.0


In [18]:
pivoted_df_county["CountyFIPS"] = pd.to_numeric(pivoted_df_county["CountyFIPS"], errors='coerce')
pivoted_df_census["LocationID"] = pd.to_numeric(pivoted_df_census["LocationID"], errors='coerce')

In [19]:
df_places_counties = pivoted_df_county
df_places_census = pivoted_df_census

# Census Age, Sex, Race, Income

In [74]:
# df_acs = pd.read_csv('data/df_acs_combined.csv') # does not match at census tract level
df_acs = pd.read_csv('data/ACSDP5Y2020_demo_housing-Data.csv', skiprows=[0])

In [75]:
df_acs.shape

(36568, 359)

In [77]:
df_acs.head(3)

,Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Margin of Error!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),...,"Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or more races!!Two races excluding Some other race, and Three or more races",Percent!!Total housing units,Percent Margin of Error!!Total housing units,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female",Unnamed: 358
0,0600000US0100190171,"Autaugaville CCD, Autauga County, Alabama",3497,534,1698,286,1799,378,94.4,21.5,...,0.6,(X),(X),2689,(X),52.5,6.2,47.5,6.2,NaN
1,0600000US0100190315,"Billingsley CCD, Autauga County, Alabama",2904,479,1367,332,1537,287,88.9,24.4,...,4.0,(X),(X),2360,(X),49.4,6.2,50.6,6.2,NaN
2,0600000US0100192106,"Marbury CCD, Autauga County, Alabama",6247,613,3127,385,3120,406,100.2,16.2,...,2.6,(X),(X),4825,(X),51.7,4.5,48.3,4.5,NaN


In [80]:
df_acs[['US_ID','LocationID']] = df_acs['Geography'].str.split('S', expand=True)

In [83]:
df_acs[["Geography",'US_ID','LocationID']].head(3)

,Geography,US_ID,LocationID
0,0600000US0100190171,0600000U,0100190171
1,0600000US0100190315,0600000U,0100190315
2,0600000US0100192106,0600000U,0100192106


In [86]:
# Filter for rows where the value is not "(X)"
df_acs = df_acs[~df_acs.eq('(X)').any(axis=1)]  # Check all columns (axis=1)

In [87]:
df_acs = df_acs.drop(['US_ID', 'Geography', 'Geographic Area Name'], axis = 1)

In [89]:
def convert_to_numeric(col):
  try:
    return pd.to_numeric(col, errors='coerce')  # Convert to NA on errors
  except:
    return col  # Keep original value if other errors occur

# df_acs_county = df_acs_county.apply(convert_to_numeric)
df_acs_census = df_acs_census.apply(convert_to_numeric)

In [90]:
# df_acs_county = df_acs_county.groupby("CountyID").mean().reset_index()
# df_acs_county.head()

In [91]:
df_acs_census = df_acs_census.groupby("LocationID").mean().reset_index()
df_acs_census.head()

,LocationID,Estimate_Households_Total,"Estimate_Households_Total_Less than $10,000","Estimate_Households_Total_$10,000 to $14,999","Estimate_Households_Total_$15,000 to $24,999","Estimate_Households_Total_$25,000 to $34,999","Estimate_Households_Total_$35,000 to $49,999","Estimate_Households_Total_$50,000 to $74,999","Estimate_Households_Total_$75,000 to $99,999","Estimate_Households_Total_$100,000 to $149,999",...,Percent_RACE_Total population_One race_Asian_Korean,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race)
0,100190171,1461.0,13.6,21.6,10.6,11.3,15.1,14.8,5.7,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1
1,100190315,1157.0,10.9,4.7,25.8,9.8,5.0,11.8,7.0,13.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.1
2,100192106,2320.0,6.7,1.5,11.2,9.3,21.0,13.4,12.9,12.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8
3,100192628,16621.0,5.1,3.5,11.7,8.1,11.8,18.0,14.6,18.2,...,0.5,0.3,0.0,0.1,0.0,0.1,0.0,0.0,0.8,3.3
4,100390207,9122.0,5.1,7.9,9.6,13.8,14.8,14.3,12.9,11.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,2.1


In [92]:
# df_acs_county.rename(columns={"CountyID":"CountyFIPS"}, inplace=True)

In [93]:
# df_acs_county["CountyFIPS"] = pd.to_numeric(df_acs_county["CountyFIPS"], errors='coerce')
df_acs_census["LocationID"] = pd.to_numeric(df_acs_census["LocationID"], errors='coerce')

## CDC Deaths

In [30]:
df_cdc = pd.read_csv('data/cdc/Underlying Cause of Death, 2018-2021, Single Race (version 1).csv')

In [31]:
df_cdc.shape

(8244, 12)

In [32]:
df_cdc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8244 entries, 0 to 8243
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Notes                            0 non-null      float64
 1   County                           8244 non-null   object 
 2   County Code                      8244 non-null   int64  
 3   Drug/Alcohol Induced             8244 non-null   object 
 4   Drug/Alcohol Induced Code        8244 non-null   object 
 5   Drug/Alcohol Induced Cause       8244 non-null   object 
 6   Drug/Alcohol Induced Cause Code  8244 non-null   object 
 7   Deaths                           8244 non-null   int64  
 8   Population                       8244 non-null   int64  
 9   Crude Rate                       8244 non-null   object 
 10  pct_tot                          8244 non-null   float64
 11  Years                            8244 non-null   object 
dtypes: float64(2), int64

In [33]:
total = df_cdc.groupby(['County Code']).sum().reset_index()
total

,County Code,Notes,County,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,pct_tot,Years
0,1001,0.0,"Autauga County, ALAutauga County, ALAutauga Co...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,2434,680130,8.8Unreliable1057.7,0.010736,2018-20212018-20212018-2021
1,1003,0.0,"Baldwin County, ALBaldwin County, ALBaldwin Co...",Drug-induced causesDrug-induced causesAlcohol-...,DDAO,Drug poisonings (overdose) Unintentional (X40-...,D1D2A9O9,10408,3639348,20.3Unreliable15.11106.7,0.011439,2018-20212018-20212018-20212018-2021
2,1005,0.0,"Barbour County, AL",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,1385,99120,1397.3,0.013973,2018-2021
3,1007,0.0,"Bibb County, ALBibb County, AL",Drug-induced causesAll other non-drug and non-...,DO,Drug poisonings (overdose) Unintentional (X40-...,D1O9,1163,178814,Unreliable1280.7,0.013008,2018-20212018-2021
4,1009,0.0,"Blount County, ALBlount County, ALBlount Count...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,3081,697758,16.810.31297.6,0.013247,2018-20212018-20212018-2021
...,...,...,...,...,...,...,...,...,...,...,...,...
3134,56037,0.0,"Sweetwater County, WYSweetwater County, WYSwee...",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,1463,509043,18.326.5817.4,0.008622,2018-20212018-20212018-2021
3135,56039,0.0,"Teton County, WY",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,377,93617,402.7,0.004027,2018-2021
3136,56041,0.0,"Uinta County, WYUinta County, WYUinta County, WY",Drug-induced causesAlcohol-induced causesAll o...,DAO,Drug poisonings (overdose) Unintentional (X40-...,D1A9O9,673,244125,UnreliableUnreliable787.7,0.008270,2018-20212018-20212018-2021
3137,56043,0.0,"Washakie County, WY",All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,383,31155,1229.3,0.012293,2018-2021


In [34]:
cdc_main = total[["County Code", "Deaths", "Population", "pct_tot"]]
cdc_main

,County Code,Deaths,Population,pct_tot
0,1001,2434,680130,0.010736
1,1003,10408,3639348,0.011439
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.013008
4,1009,3081,697758,0.013247
...,...,...,...,...
3134,56037,1463,509043,0.008622
3135,56039,377,93617,0.004027
3136,56041,673,244125,0.008270
3137,56043,383,31155,0.012293


In [35]:
cdc_main['pct_tot'] = cdc_main['Deaths']/cdc_main['Population']
cdc_main

,County Code,Deaths,Population,pct_tot
0,1001,2434,680130,0.003579
1,1003,10408,3639348,0.002860
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.006504
4,1009,3081,697758,0.004416
...,...,...,...,...
3134,56037,1463,509043,0.002874
3135,56039,377,93617,0.004027
3136,56041,673,244125,0.002757
3137,56043,383,31155,0.012293


In [36]:
col_names = ["CountyFIPS", "DrugOrAlc_Deaths", "Population", "DrugOrAlc_Deaths_pct"]
cdc_main.columns = col_names
cdc_main.head()

,CountyFIPS,DrugOrAlc_Deaths,Population,DrugOrAlc_Deaths_pct
0,1001,2434,680130,0.003579
1,1003,10408,3639348,0.002860
2,1005,1385,99120,0.013973
3,1007,1163,178814,0.006504
4,1009,3081,697758,0.004416


In [37]:
df_cdc_counties = cdc_main[["CountyFIPS", "DrugOrAlc_Deaths_pct"]]

In [38]:
df_cdc_counties["CountyFIPS"] = pd.to_numeric(df_cdc_counties["CountyFIPS"], errors='coerce')

## FTE Law Enforcement Officers

In [39]:
df_le = pd.read_excel('data/fbi_le/FTE_Law_Enforcement_Officers.xlsx', skiprows=[0,1,2])

In [40]:
df_le.head()

,State,County,Total law\nenforcement\nemployees,Total\nofficers,Total\ncivilians
0,ALABAMA-Metropolitan Counties,Autauga,43.0,32.0,11.0
1,NaN,Baldwin,319.0,135.0,184.0
2,NaN,Bibb,12.0,11.0,1.0
3,NaN,Blount,48.0,44.0,4.0
4,NaN,Colbert,57.0,31.0,26.0


In [41]:
df_hpsa_pc.columns

Index(['HPSA Name', 'HPSA ID', 'Designation Type', 'HPSA Discipline Class',
       'HPSA Score', 'PC MCTA Score', 'Primary State Abbreviation',
       'HPSA Status', 'HPSA Designation Date',
       'HPSA Designation Last Update Date', 'Metropolitan Indicator',
       'LocationID', 'HPSA Degree of Shortage', 'Withdrawn Date', 'HPSA FTE',
       'HPSA Designation Population', '% of Population Below 100% Poverty',
       'HPSA Formal Ratio', 'HPSA Population Type', 'Rural Status',
       'Longitude', 'Latitude', 'BHCMIS Organization Identification Number',
       'Break in Designation', 'Common County Name', 'Common Postal Code',
       'Common Region Name', 'Common State Abbreviation', 'CountyFIPS',
       'Common State FIPS Code', 'Common State Name', 'County Equivalent Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Discipline Class Number', 'HPSA Address', 'HPSA City',
       'HPSA Component Name', 'HPSA Component Source Identification

In [42]:
df_le = df_le.groupby("County")["Total law\nenforcement\nemployees"].sum().reset_index()

In [43]:
df_le.head()

,County,Total law\nenforcement\nemployees
0,Abbeville,656.0
1,Acadia,104.0
2,Accomack,64.0
3,Ada,720.0
4,Adair,72.0


In [44]:
counties = df_hpsa_pc[["County Equivalent Name","Common County Name","CountyFIPS"]].drop_duplicates()

In [45]:
counties.rename(columns={"County Equivalent Name":"County"}, inplace=True)
counties.head()

,County,Common County Name,CountyFIPS
0,Lucas,"Lucas County, OH",39095
5,Cuyahoga,"Cuyahoga County, OH",39035
16,Van Wert,"Van Wert County, OH",39161
24,Mahoning,"Mahoning County, OH",39099
52,Huron,"Huron County, OH",39077


In [46]:
df_le_counties = df_le.merge(counties, on="County")

In [47]:
df_le_counties = df_le_counties[["CountyFIPS", "Total law\nenforcement\nemployees"]]

In [48]:
df_le_counties.rename(columns={"Total law\nenforcement\nemployees":"tot_law_enforce_FTE"}, inplace=True)

In [49]:
df_le_counties

,CountyFIPS,tot_law_enforce_FTE
0,45001,656.0
1,22001,104.0
2,51001,64.0
3,21001,72.0
4,40001,72.0
...,...,...
2304,48503,34.0
2305,06115,163.0
2306,04027,352.0
2307,08125,352.0


In [50]:
df_le_counties["CountyFIPS"] = pd.to_numeric(df_le_counties["CountyFIPS"], errors='coerce')

# Final Dataset

In [51]:
# at county level
combined_df = df_hpsa_county_scores.merge(df_places_counties, how="inner", on="CountyFIPS")
combined_df = combined_df.merge(df_le_counties, how="inner", on="CountyFIPS")
combined_df = combined_df.merge(df_acs_county, how="inner", on="CountyFIPS")
final_df_counties = combined_df.merge(df_cdc_counties, how="inner", on="CountyFIPS")

# at census level
df_hpsa_places_census = df_hpsa_census_scores.merge(df_places_census, how='inner', on ="LocationID")
print(df_hpsa_places_census.shape)
df_hpsa_acs_census = df_hpsa_census_scores.merge(df_acs_census, how='inner', on ="LocationID")
print(df_hpsa_acs_census.shape)

(11653, 39)
(1799, 54)


In [52]:
final_df_counties.head()

,CountyFIPS,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,...,Percent_RACE_Total population_One race_Asian_Vietnamese,Percent_RACE_Total population_One race_Asian_Other Asian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Native Hawaiian,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Chamorro,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Samoan,Percent_RACE_Total population_One race_Native Hawaiian and Other Pacific Islander_Other Pacific Islander,Percent_RACE_Total population_One race_Some other race,Percent_HISPANIC OR LATINO AND RACE_Total population_Hispanic or Latino (of any race),DrugOrAlc_Deaths_pct
0,1001,15.0,11.100000,30.291667,14.958333,39.291667,79.225000,6.458333,10.483333,83.750000,...,0.0750,0.00,0.025000,0.000000,0.025,0.0,0.0,0.350000,1.825000,0.003579
1,1003,14.0,9.458065,31.293548,16.025806,37.838710,80.809677,7.670968,9.929032,84.980645,...,0.0375,0.05,0.000000,0.000000,0.000,0.0,0.0,2.125000,5.637500,0.002860
2,1005,20.0,17.244444,34.655556,12.111111,47.322222,81.455556,6.755556,11.755556,80.944444,...,0.0600,0.06,0.020000,0.020000,0.000,0.0,0.0,2.480000,4.360000,0.013973
3,1007,10.0,13.875000,31.125000,15.125000,40.075000,78.750000,6.475000,10.575000,82.050000,...,0.0000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.033333,1.966667,0.006504
4,1009,7.0,13.122222,31.922222,15.011111,38.544444,79.044444,7.200000,10.400000,82.400000,...,0.0000,0.00,0.128571,0.028571,0.100,0.0,0.0,1.771429,10.871429,0.004416


In [53]:
df = df_hpsa_places_census

def categorize_score(score):
  if score < 14:
    return 0
  else:
    return 1

df['HPSA Category'] = df['HPSA Score'].apply(categorize_score)
df.head()

,LocationID,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,...,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION,HPSA Category
0,4001942600,11.0,20.3,27.9,13.4,37.8,67.3,5.9,15.0,68.4,...,26.1,27.7,41.7,22.6,10.8,40.8,7.0,37.5,15.4,0
1,4001942700,11.0,17.3,26.7,14.2,35.2,66.5,6.1,14.6,71.8,...,24.1,24.2,38.9,20.0,8.6,39.3,6.2,30.8,12.3,0
2,4001944000,11.0,15.8,25.3,14.6,33.6,65.4,5.6,14.2,72.4,...,23.7,22.0,38.8,18.7,7.9,39.2,5.4,27.7,11.2,0
3,4001944100,11.0,19.0,24.7,14.5,33.1,63.0,5.1,15.7,68.5,...,27.6,23.9,40.3,20.8,9.5,40.5,5.8,33.9,13.9,0
4,4001944201,11.0,19.2,25.0,14.0,33.0,64.0,5.3,15.5,70.6,...,26.6,24.0,39.7,20.5,9.4,39.8,5.7,34.4,13.7,0


In [54]:
# df.to_csv('data/final_data_mean.csv')
df.to_csv('data/df_hpsa_places_census.csv')

# Building our model

In [55]:
# df = pd.read_csv("data/final_data_mean.csv")

In [56]:
df.head()

,LocationID,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,...,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION,HPSA Category
0,4001942600,11.0,20.3,27.9,13.4,37.8,67.3,5.9,15.0,68.4,...,26.1,27.7,41.7,22.6,10.8,40.8,7.0,37.5,15.4,0
1,4001942700,11.0,17.3,26.7,14.2,35.2,66.5,6.1,14.6,71.8,...,24.1,24.2,38.9,20.0,8.6,39.3,6.2,30.8,12.3,0
2,4001944000,11.0,15.8,25.3,14.6,33.6,65.4,5.6,14.2,72.4,...,23.7,22.0,38.8,18.7,7.9,39.2,5.4,27.7,11.2,0
3,4001944100,11.0,19.0,24.7,14.5,33.1,63.0,5.1,15.7,68.5,...,27.6,23.9,40.3,20.8,9.5,40.5,5.8,33.9,13.9,0
4,4001944201,11.0,19.2,25.0,14.0,33.0,64.0,5.3,15.5,70.6,...,26.6,24.0,39.7,20.5,9.4,39.8,5.7,34.4,13.7,0


In [57]:
df.shape

(11653, 40)

In [58]:
df.dropna(axis=0,inplace=True)
df.shape

(10503, 40)

In [59]:
df.head()

,LocationID,HPSA Score,ACCESS2,ARTHRITIS,BINGE,BPHIGH,BPMED,CANCER,CASTHMA,CERVICAL,...,MHLTH,MOBILITY,OBESITY,PHLTH,SELFCARE,SLEEP,STROKE,TEETHLOST,VISION,HPSA Category
0,4001942600,11.0,20.3,27.9,13.4,37.8,67.3,5.9,15.0,68.4,...,26.1,27.7,41.7,22.6,10.8,40.8,7.0,37.5,15.4,0
1,4001942700,11.0,17.3,26.7,14.2,35.2,66.5,6.1,14.6,71.8,...,24.1,24.2,38.9,20.0,8.6,39.3,6.2,30.8,12.3,0
2,4001944000,11.0,15.8,25.3,14.6,33.6,65.4,5.6,14.2,72.4,...,23.7,22.0,38.8,18.7,7.9,39.2,5.4,27.7,11.2,0
3,4001944100,11.0,19.0,24.7,14.5,33.1,63.0,5.1,15.7,68.5,...,27.6,23.9,40.3,20.8,9.5,40.5,5.8,33.9,13.9,0
4,4001944201,11.0,19.2,25.0,14.0,33.0,64.0,5.3,15.5,70.6,...,26.6,24.0,39.7,20.5,9.4,39.8,5.7,34.4,13.7,0


In [60]:
# features = df.drop(['CountyFIPS', 'HPSA Score', 'HPSA Category'], axis=1)
features = df.drop(['LocationID', 'HPSA Score', 'HPSA Category'], axis=1)
target = df['HPSA Category']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.25, stratify=target, random_state=42)

ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

recall_scores = []

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_val_scaled = scaler.transform(X_val)

models = [LogisticRegression(),
          DecisionTreeClassifier(),
          RandomForestClassifier(),
#           KNeighborsClassifier(),
          SVC(kernel='rbf', probability=True),
          XGBClassifier()]

for model in models:
    model.fit(X_train_scaled, Y_resampled)
    print(f'{model} : ')

    # Optional: Calculate and print training and validation ROC AUC scores
    # train_preds_proba = model.predict_proba(X_train_scaled)[:, 1]
    # print('Training ROC AUC Score : ', roc_auc_score(y_train, train_preds_proba))
    # val_preds_proba = model.predict_proba(X_val_scaled)[:, 1]
    # print('Validation ROC AUC Score : ', roc_auc_score(y_val, val_preds_proba))

    train_preds = model.predict(X_train_scaled)
    print('Training Accuracy : ', accuracy_score(Y_resampled, train_preds))

    val_preds = model.predict(X_val_scaled)
    print('Validation Accuracy : ', accuracy_score(y_val, val_preds))
    print()

    conf_matrix = confusion_matrix(y_val, val_preds, normalize='true')  # Normalized confusion matrix

    print("Confusion Matrix:")
    print(conf_matrix)

#     test_recall = recall_score(y_val, val_preds)
#     recall_scores.append(test_recall)
#     print(f"Recall: {test_recall}")

    print("")
    print("----------------------")
    print("")

LogisticRegression() : 
Training Accuracy :  0.7335812772133526
Validation Accuracy :  0.7372429550647372

Confusion Matrix:
[[0.74873096 0.25126904]
 [0.26768226 0.73231774]]

----------------------

DecisionTreeClassifier() : 
Training Accuracy :  1.0
Validation Accuracy :  0.7741812642802742

Confusion Matrix:
[[0.61040609 0.38959391]
 [0.15560392 0.84439608]]

----------------------

RandomForestClassifier() : 
Training Accuracy :  1.0
Validation Accuracy :  0.8396801218583397

Confusion Matrix:
[[0.67639594 0.32360406]
 [0.09031556 0.90968444]]

----------------------

SVC(probability=True) : 
Training Accuracy :  0.8377177068214804
Validation Accuracy :  0.8206397562833206

Confusion Matrix:
[[0.81979695 0.18020305]
 [0.17899891 0.82100109]]

----------------------

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
        

In [61]:
features = df.drop(['LocationID', 'HPSA Score', 'HPSA Category'], axis=1)
target = df['HPSA Category']

X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.2, stratify=target, random_state=42)

ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_val_scaled = scaler.transform(X_val)

lr_model = LogisticRegression()







lr_model.fit(X_train_scaled, Y_resampled)

# Retrieve the coefficients and feature names
coefficients = lr_model.coef_[0]
feature_names = X_train.columns

# Create a DataFrame to display the coefficients and feature names
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Odds Ratio': np.exp(coefficients)
})

# Sort the DataFrame by the absolute value of coefficients to highlight the most impactful features
feature_importance_df = feature_importance_df.reindex(feature_importance_df['Coefficient'].abs().sort_values(ascending=False).index)

# Display the feature importance DataFrame
print("Feature Importance:")
feature_importance_df.head(10)

Feature Importance:


,Feature,Coefficient,Odds Ratio
29,MOBILITY,3.624782,37.516545
11,COGNITION,-3.514447,0.029764
34,STROKE,-2.767551,0.062816
28,MHLTH,1.533310,4.633490
17,DENTAL,-1.478647,0.227946
12,COLON_SCREEN,1.369442,3.933154
5,CANCER,0.923969,2.519270
18,DEPRESSION,-0.894634,0.408757
22,HEARING,-0.876076,0.416414
10,CHOLSCREEN,-0.859408,0.423413


# Ridge Regression

In [62]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error  

In [63]:
X = df.drop(['LocationID', 'HPSA Score', 'HPSA Category'], axis=1)
y = df['HPSA Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
# Define a range of alpha values to explore
alpha_range = np.logspace(-3, 2, 6)  # Example: explore alpha from 0.001 to 100

param_grid = {'alpha': alpha_range}

In [65]:
ridge_reg = Ridge()  # Create the base model

grid_search = GridSearchCV(estimator=ridge_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')  # Use negative MSE for minimization
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring='neg_mean_squared_error')

In [66]:
best_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']

print(f"Best Alpha: {best_alpha}")

Best Alpha: 1.0


In [67]:
alpha = 1.0  

ridge_reg = Ridge(alpha=alpha)
ridge_reg.fit(X_train, y_train)

Ridge()

In [68]:
y_pred = ridge_reg.predict(X_test)

In [69]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 8.303213619268202
